# Merging Airline Delay and Weather Datasets

In this notebook, we merge together two data sources in order to create richer features for our flight delay prediction classification problem.
* selecting the columns we wish to keep for later analysis
* converting and cleaning data where required
* handling missing values

#### Import required modules

Import and configure the required modules.

In [1]:
# !pip install pandas scikit-learn > /dev/null 2>&1

In [2]:
# Define required imports
import pandas as pd
# These set pandas max column and row display in the notebook
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

### Read datasets

We start by reading in the processed flight delay and weather datasets

In [3]:
flight_path = 'data/jfk_flight_features.csv'
flight_data = pd.read_csv(flight_path, parse_dates=['flight_date'])
flight_data.head()

,flight_date,month,day_of_month,day_of_week,airline_id,airline_name,origin,dest,sched_dep_time,dep_time_bin,distance_bin,delayed,dep_delay
0,2015-03-17,3,17,2,20409,JetBlue,JFK,RSW,1525,1500-1559,5,0,-7
1,2016-05-25,5,25,3,20409,JetBlue,JFK,SJU,2119,2100-2159,7,0,0
2,2017-07-26,7,26,3,19805,American Airlines,JFK,SFO,700,0700-0759,11,0,-3
3,2010-12-11,12,11,6,20409,JetBlue,JFK,LAS,1900,1900-1959,9,0,-5
4,2012-08-11,8,11,6,20398,Envoy Air,JFK,IND,1510,1500-1559,3,0,-2


In [4]:
weather_path = 'data/jfk_weather_features.csv'
weather_data = pd.read_csv(weather_path, parse_dates=['DATE'])
weather_data.head()

,DATE,visibility,weather_type_raw,wind_speed,wind_gust_speed,precip,rain,ice_pellets,mist,snow,drizzle,haze,fog,thunderstorm,smoke,unknown_precipitation
0,2010-01-01 00:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0,0,0.01,1,1,1,0,0,0,0,0,0,0
1,2010-01-01 01:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0,0,0.02,1,1,1,0,0,0,0,0,0,0
2,2010-01-01 02:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0,0,0.00,1,0,1,0,0,0,0,0,0,0
3,2010-01-01 03:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0,0,0.00,1,0,1,0,0,0,0,0,0,0
4,2010-01-01 04:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0,0,0.00,1,0,1,0,0,0,0,0,0,0


### Merge datasets

The next step is to merge or join the two datasets, such that for each flight record in the flight delay dataset, we have information about the weather conditions present for that flight. 

**Note** we have to be careful not to effectively "leak" information. Recall that our weather observations come from automated weather station reports that are generated on the 51st minute of each hour. We must ensure that the weather report used for flight delay prediction is one covering weather conditions present _before_ the flight departure, otherwise we would be giving our model a glimpse in the the future!

This makes joining the datasets a little tricky. One simple approach is to join the record for a given flight day and hour, with the weather reading for the same day but the _previous hour_. We can do this by extracting 2 "join keys" from each dataset: the first for the `date` and the second for the `hour` of the record. If we set the `hour` join key for the flight to the hour _before_ the actual hour of the flight scheduled departure, then we ensure the corresponding weather report comes from the hour before the flight would depart.

In [5]:
flight_data.loc[:, 'hour_key'] = pd.to_datetime(flight_data['sched_dep_time'], format='%H%M', errors='ignore').dt.hour - 1
flight_data.loc[:, 'date_key'] = flight_data['flight_date'].dt.date
flight_data.head()

,flight_date,month,day_of_month,day_of_week,airline_id,airline_name,origin,dest,sched_dep_time,dep_time_bin,distance_bin,delayed,dep_delay,hour_key,date_key
0,2015-03-17,3,17,2,20409,JetBlue,JFK,RSW,1525,1500-1559,5,0,-7,14,2015-03-17
1,2016-05-25,5,25,3,20409,JetBlue,JFK,SJU,2119,2100-2159,7,0,0,20,2016-05-25
2,2017-07-26,7,26,3,19805,American Airlines,JFK,SFO,700,0700-0759,11,0,-3,6,2017-07-26
3,2010-12-11,12,11,6,20409,JetBlue,JFK,LAS,1900,1900-1959,9,0,-5,18,2010-12-11
4,2012-08-11,8,11,6,20398,Envoy Air,JFK,IND,1510,1500-1559,3,0,-2,14,2012-08-11


In [6]:
weather_data.loc[:, 'date_key'] = weather_data['DATE'].dt.date
weather_data.loc[:, 'hour_key'] = weather_data['DATE'].dt.hour
weather_data.head()

,DATE,visibility,weather_type_raw,wind_speed,wind_gust_speed,precip,rain,ice_pellets,mist,snow,drizzle,haze,fog,thunderstorm,smoke,unknown_precipitation,date_key,hour_key
0,2010-01-01 00:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0,0,0.01,1,1,1,0,0,0,0,0,0,0,2010-01-01,0
1,2010-01-01 01:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0,0,0.02,1,1,1,0,0,0,0,0,0,0,2010-01-01,1
2,2010-01-01 02:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0,0,0.00,1,0,1,0,0,0,0,0,0,0,2010-01-01,2
3,2010-01-01 03:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0,0,0.00,1,0,1,0,0,0,0,0,0,0,2010-01-01,3
4,2010-01-01 04:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0,0,0.00,1,0,1,0,0,0,0,0,0,0,2010-01-01,4


Next, we join the datasets together based on the "join keys" we have created:

In [7]:
flight_weather_data = flight_data.merge(weather_data, how='inner', on=['date_key', 'hour_key'])
flight_weather_data.head()

,flight_date,month,day_of_month,day_of_week,airline_id,airline_name,origin,dest,sched_dep_time,dep_time_bin,distance_bin,delayed,dep_delay,hour_key,date_key,DATE,visibility,weather_type_raw,wind_speed,wind_gust_speed,precip,rain,ice_pellets,mist,snow,drizzle,haze,fog,thunderstorm,smoke,unknown_precipitation
0,2015-03-17,3,17,2,20409,JetBlue,JFK,RSW,1525,1500-1559,5,0,-7,14,2015-03-17,2015-03-17 14:51:00,10.0,NaN,30,9,0.0,0,0,0,0,0,0,0,0,0,0
1,2016-05-25,5,25,3,20409,JetBlue,JFK,SJU,2119,2100-2159,7,0,0,20,2016-05-25,2016-05-25 20:51:00,10.0,NaN,11,0,0.0,0,0,0,0,0,0,0,0,0,0
2,2017-07-26,7,26,3,19805,American Airlines,JFK,SFO,700,0700-0759,11,0,-3,6,2017-07-26,2017-07-26 06:51:00,10.0,NaN,7,0,0.0,0,0,0,0,0,0,0,0,0,0
3,2010-12-11,12,11,6,20409,JetBlue,JFK,LAS,1900,1900-1959,9,0,-5,18,2010-12-11,2010-12-11 18:51:00,10.0,NaN,3,0,0.0,0,0,0,0,0,0,0,0,0,0
4,2012-08-11,8,11,6,20398,Envoy Air,JFK,IND,1510,1500-1559,3,0,-2,14,2012-08-11,2012-08-11 14:51:00,10.0,NaN,18,0,0.0,0,0,0,0,0,0,0,0,0,0


For the first record in our flight dataset, we can see that the flight departs at 15:25. The corresponding weather report is timestamped at 14:51.

**Note** all we guarantee here is that the weather report is _within_ 1 hour before the flight departure, not _precisely 1 hour before_. 

### Save the Merged Data

Finally, we save the merged dataset for use by downstream tasks.

In [8]:
flight_weather_data.to_csv('data/jfk_flight_weather_features.csv', index=False, float_format='%g')

<a id="authors"></a> 
### Authors

This notebook was created by the [Center for Open-Source Data & AI Technologies](http://codait.org).

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.